# Imports and Setups

In [ ]:
! pip install transformers==4.10.1

     |████████████████████████████████| 2.8 MB 8.3 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 3.3 MB 32.1 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! pip install nlpaug==1.1.7

     |████████████████████████████████| 405 kB 8.5 MB/s 


In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 26.1 MB/s 


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 5.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0e4b1fdd7fe1ee539b7d1c52b96e19264d01d0e9a36269475d053d7cf73b2722
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-7u3sv0if
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-7u3sv0if
     |████████████████████████████████| 50 kB 5.6 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=c7cb6afb76fb4ea5af1266c483393b1dc6dfded3a897fe400f54d4c24acb9038
  Stored in directory: /tmp/pip-ephem-wheel-cache-epzrbach/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149866 sha256=a16d07b279c045a5d1acda3ef5ca5359704332ed0bd7d26b77db103373969e4e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built parrot python-Levenshtein


Lendo o Arquivo

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/semeval_ironic/SemEval2018-T3-train-taskA_emoji.txt',sep = '\t')
dataset.head()

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
3,4,0,3 episodes left I'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...


In [ ]:
num_gen = 5
perc = 100

In [ ]:
X = base_cru_1p['text']
y = base_cru_1p['label']

In [ ]:
base_cru_1p = dataset.copy()

Gerando Base Percentual

In [ ]:
from sklearn.model_selection import train_test_split
X_resto, X_treino_1p, y_resto, y_treino_1p = train_test_split(X, y, stratify=y, test_size=(perc/100), random_state=42) # Selecionando 1%

In [ ]:
import pandas as pd

data = {'texto': X_treino_1p, 'label': y_treino_1p}
base_cru_1p = pd.DataFrame.from_dict(data)

In [ ]:
import pandas as pd

data = {'texto': X, 'label': y}
base_cru_1p = pd.DataFrame.from_dict(data)

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p.head()

,texto,label
0,im feeling quite sad and sorry for myself but ...,0
1,i feel like i am still looking at a blank canv...,0
2,i feel like a faithful servant,2
3,i am just feeling cranky and blue,3
4,i can have for a treat or if i am feeling festive,1


In [ ]:
base_cru_1p.shape

(2000, 2)

Só rodar quando gerar o original

In [ ]:
repeticoes = 1

In [ ]:
# arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv'
# base_cru_1p.to_csv(arq, encoding='utf-8', index=False, sep=';', decimal = ',')

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

# Embedding Word2Vec


In [ ]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_glove('glove.6B', '.')

In [ ]:
aug_emb = naf.Sequential([naf.Sometimes([
    naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="substitute")]),
    naf.Sometimes([naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="insert"),
])])

In [ ]:
exp_emb = []

for i in range(repeticoes):
  base_cru_1p_emb = base_cru_1p.copy()
  base_cru_1p_emb['parafraseador'] = base_cru_1p_emb['texto'].progress_apply(lambda x:aug_emb.augment(x,num_gen))
  base_cru_1p_emb = base_cru_1p_emb.explode('parafraseador').reset_index(drop=True)
  exp_emb.append(base_cru_1p_emb)

  arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv'
  exp_emb[i].to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 2000/2000 [1:11:31<00:00,  2.15s/it]


In [1]:
1/2.15

0.46511627906976744